In [19]:
# balance chunks
max_tokens = 128

# LLM max tokens output
max_tokens_output = 256

top_k_bm25 = 5

In [ ]:
from langchain.prompts import PromptTemplate
from langchain_community.chat_models import ChatOpenAI
from langchain.schema.retriever import BaseRetriever
from rank_bm25 import BM25Okapi
from pyvi import ViTokenizer
import re
import os
from langchain.schema.document import Document
from typing import List
from pydantic import PrivateAttr
from langchain.tools import DuckDuckGoSearchRun
from langchain.agents import initialize_agent, Tool
from langchain.agents.agent_types import AgentType
from langchain.memory import ConversationBufferMemory
from langchain.schema import HumanMessage, AIMessage
from langchain.chains import ConversationChain
from langchain.memory import ConversationSummaryMemory


# Go llama.com to take api_key
# Load LLM
def load_llm(max_tokens_output=512):
    llm = ChatOpenAI(
        model_name="meta-llama/Llama-3.3-70B-Instruct-Turbo-Free",
        openai_api_key="YOUR_API_KEY",
        openai_api_base="https://api.together.xyz",
        temperature=0.01,
        max_tokens= max_tokens_output,
    )
    return llm

def tokenize(text):
    return ViTokenizer.tokenize(text).split()

def detokenize(tokens):
    return ' '.join(tokens).replace('_', ' ')

def load_all_text_chunks(folderpath, max_tokens=max_tokens):
    all_chunks = []

    for filename in os.listdir(folderpath):
        if filename.endswith(".txt"):
            filepath = os.path.join(folderpath, filename)
            with open(filepath, "r", encoding="utf-8") as f:
                text = f.read()

            # Step 1: Separate by pattern Điều X.
            raw_chunks = re.split(r"(?=Điều\s\d+[.,])", text)
            raw_chunks = [chunk.strip() for chunk in raw_chunks if chunk.strip()]
            name_without_ext = os.path.splitext(filename)[0]

            for idx, chunk in enumerate(raw_chunks, start=1):
                # Xu ly tung dieu mot
                tokens = tokenize(chunk)
                sub_chunks = []
                if len(tokens) > max_tokens:
                    # Neu chunk qua dai, chia ra thanh nhieu chunk nho hon
                    # Chia chunk thanh nhieu phan nho hon
                    for i in range(0, len(tokens), max_tokens):
                        sub_chunk = tokens[i:i + max_tokens]
                        sub_chunk_text = detokenize(sub_chunk)
                        sub_chunks.append(sub_chunk_text)
                    if len(tokens) % max_tokens != 0:
                        # Neu phan du khong du max_tokens, them phan con lai vao chunk cuoi cung
                        sub_chunks[-1] += " " + detokenize(tokens[len(tokens) - (len(tokens) % max_tokens):])
                
                else:
                    sub_chunks = [chunk]

                # Them cac chunk da chia vao danh sach all_chunks
                for sub_idx, sub_chunk in enumerate(sub_chunks, start=1):
                    chunk_data = {
                            "global_id": len(all_chunks),
                            "id": f"{name_without_ext}_{idx}_{sub_idx}",
                            "filename": filename,
                            "sourcename":f"{idx}",
                            "content": sub_chunk
                    }
                    all_chunks.append(chunk_data)
    return all_chunks

# Tokenize all chunks
def tokenize_chunks(chunks):
    return [ViTokenizer.tokenize(chunk['content']).split() for chunk in chunks]

# Custom BM25 Retriever
class CustomBM25Retriever(BaseRetriever):
    _bm25: BM25Okapi = PrivateAttr()
    _tokenized_chunks: List[List[str]] = PrivateAttr()
    _original_chunks: List[dict] = PrivateAttr()
    _k: int = PrivateAttr()

    def __init__(self, bm25, tokenized_chunks, original_chunks, top_k_bm25 =6):
        super().__init__()
        self._bm25 = bm25
        self._tokenized_chunks = tokenized_chunks
        self._original_chunks = original_chunks
        self._bm25_k = top_k_bm25

    def _get_relevant_documents(self, query: str) -> List[Document]:
        tokenized_query = ViTokenizer.tokenize(query).split()
        scores = self._bm25.get_scores(tokenized_query)
        top_indices = sorted(range(len(scores)), key=lambda i: -scores[i])[:self._bm25_k]

        docs = []
        for i in top_indices:
            chunk = self._original_chunks[i]
            doc = Document(
                page_content=chunk["content"],
                metadata={
                    "global_id": i, # index of chunk in all_chunks
                    "file_name_id": chunk["id"], # id of chunk in file
                    "filename": chunk["filename"], # name of root file
                    "sourcename": chunk["sourcename"], # name of chunk in file
                    "score": float(scores[i])
                }
            )
            docs.append(doc)

        return docs


def build_context_from_docs(docs: List[Document], original_chunks: List[dict]) -> str:
    context_blocks = []

    for doc in docs:
        idx = doc.metadata["global_id"]
        block_parts = []

        # Thêm previous chunk (nếu có)
        if idx  >= 1:
            block_parts.append(original_chunks[idx - 1]["content"])

        # Thêm current chunk
        block_parts.append(original_chunks[idx]["content"])

        # Thêm next chunk (nếu có)
        if idx + 1 < len(original_chunks):
            block_parts.append(original_chunks[idx + 1]["content"])

        # Nối thành một block duy nhất
        full_block = " ".join(block_parts)
        full_block = f"Đoạn context này thuộc điều {original_chunks[idx]['sourcename']}:" + full_block
        context_blocks.append(full_block.strip())

    return "\n\n".join(context_blocks)

template = """<|im_start|>system
Bạn là một chatbot trả lời về quy định của Đại học Bách khoa Hà Nội. Chỉ trả lời dựa trên thông tin đã cung cấp và nêu rõ dựa trên thông tin gì để trả lời. Nếu không chắc chắn hoặc không tìm thấy, hãy trả lời 'Tôi xin lỗi, tôi không có đủ thông tin để trả lời câu hỏi này.'
{context}<|im_end|>
<|im_start|>user
{question}<|im_end|>
<|im_start|>assistant"""


# Load components
llm = load_llm(max_tokens_output)

# Load BM25 Text and Tokenize
all_chunks = load_all_text_chunks("DATA/preprocessed_data")
tokenized_chunks = tokenize_chunks(all_chunks)
bm25_model = BM25Okapi(tokenized_chunks)

# Build Custom BM25 Retriever

custom_bm25_retriever = CustomBM25Retriever(
        bm25=bm25_model,
        tokenized_chunks=tokenized_chunks,
        original_chunks=all_chunks,
        top_k_bm25=top_k_bm25,
    )

In [21]:
# Assess chunk lengths 
import numpy as np

def calculate_chunk_length_stats(chunks):
    texts = [chunk["content"] for chunk in chunks]

    lengths = [len(tokenize(text)) for text in texts]
    median_length = np.median(lengths)
    mean_length = np.mean(lengths)
    variance = np.var(lengths)

    return median_length,mean_length, variance, lengths

median_length, mean_l, var, lengths = calculate_chunk_length_stats(all_chunks)
print(f"Max Length: {max(lengths)}")
print(f"Min Length: {min(lengths)}")
print(f"Median Length: {median_length}")
print(f"Mean Length: {mean_l}")
print(f"Variance: {var}")
lengths = sorted(lengths)
print(lengths)

Max Length: 234
Min Length: 6
Median Length: 127.0
Mean Length: 120.9349593495935
Variance: 1964.8738184942827
[6, 12, 16, 20, 20, 22, 28, 30, 34, 34, 36, 40, 50, 64, 68, 70, 74, 76, 78, 80, 91, 94, 96, 114, 116, 120, 120, 124, 124, 124, 125, 125, 125, 125, 125, 125, 126, 126, 126, 126, 126, 126, 126, 126, 126, 126, 126, 126, 126, 126, 126, 126, 126, 126, 126, 126, 126, 127, 127, 127, 127, 127, 127, 127, 127, 127, 127, 127, 127, 127, 127, 127, 127, 127, 127, 127, 127, 127, 127, 127, 127, 127, 127, 127, 127, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 129, 129, 129, 129, 136, 136, 146, 154, 172, 174, 178, 182, 194, 200, 210, 216, 226, 228, 230, 234, 234]


In [22]:
import csv

hangmuc = []
hangmuc_to_contact = {}
with open('contact_table.csv', 'r', encoding='utf-8') as f:
    reader = csv.reader(f)
    for row in reader:
        hangmuc.append(row[0])
        hangmuc_to_contact[row[0]] = row[1]

In [23]:
from sentence_transformers import SentenceTransformer
import torch

# Download from the 🤗 Hub
model = SentenceTransformer("hiieu/halong_embedding")
hangmuc_embeddings = model.encode(hangmuc)

def get_best_related_contact(query, model, hangmuc_embeddings, top_k = 5):
    query_embedding = model.encode([query])
    similarities = model.similarity(query_embedding, hangmuc_embeddings).flatten()

    sorted_indices = torch.argsort(similarities, descending=True)
    sorted_hangmuc = [hangmuc[idx] for idx in sorted_indices][:top_k]
    return sorted_hangmuc

In [24]:
get_best_related_contact("Thời gian học", model, hangmuc_embeddings, top_k = 10)

['Lịch thi học kỳ',
 'Thắc mắc về thời khóa biểu',
 'Tiếp nhận trở lại học sau khi nghỉ dài hạn(Thời gian tiếp nhận đơn là trước 3-4 tuần so với thời điểm bắt đầu học kỳ)',
 'Hoãn thi cuối kỳ',
 'Xin nghỉ học dài hạn (bảo lưu)',
 'Chuyển ngành đào tạo/chương trình đào tạo',
 'Biên lai học phí',
 'Lịch đăng ký học tậpThắc mắc đăng ký học tập',
 'Công tác lưu học sinh',
 'Kết quả học tập các kỳ (GPA, CPA, tín chỉ nợ...)Tăng/giảm mức cảnh báo học tập']

In [ ]:
def generate_paraphrased_questions(original_question: str, llm) -> List[str]:
    prompt = f"""
Hãy viết lại câu hỏi sau thành 2 câu hỏi khác nhau nhưng vẫn giữ nguyên nghĩa, ngắn gọn và tự nhiên, để cải thiện khả năng tìm kiếm văn bản:

Câu hỏi gốc: "{original_question}"

Các câu hỏi viết lại:
1.
2.
"""
    response = llm.invoke(prompt)
    lines = response.content.strip().split("\n")
    paraphrased = [line.split(". ", 1)[1] if ". " in line else line for line in lines if line.strip()]
    return paraphrased[-2:]

def ask_question_with_variants(question: str, retriever, conversation, template: str, all_chunks: List[dict], debug = True):
    # Tạo các biến thể của câu hỏi
    paraphrased_questions = generate_paraphrased_questions(question, llm)
    all_questions = [question] + paraphrased_questions
    if debug:
        print("Các câu hỏi đã tạo:")
        for i, q in enumerate(all_questions):
            print(f"{i + 1}. {q}")
        print("\nĐang tìm kiếm câu trả lời từ RAG...")
    
    for q in all_questions:
        docs = retriever._get_relevant_documents(q)
        context = build_context_from_docs(docs, all_chunks)

        # Lấy tóm tắt hội thoại từ memory
        memory_summary = conversation.memory.buffer  # or use .load_memory_variables({})['history']
        if memory_summary:
            context = f"Tóm tắt hội thoại trước đó:\n{memory_summary}\n\n{context}"

        prompt = template.format(context=context, question=q)
        answer = conversation.predict(input=prompt)

        if "không có đủ thông tin để trả lời câu hỏi này" not in answer.lower():
            if debug:
                print("Tìm được câu trả lời từ RAG:")
                print(f"Câu hỏi dùng: {q}")
            print(f"Câu trả lời: {answer}")
            
            if "liên hệ" in answer.lower():
                # Nếu câu hỏi liên quan đến liên hệ, tìm kiếm thông tin liên hệ
                related_contacts = get_best_related_contact(q, model, hangmuc_embeddings,5)
                print("Nếu bạn cần thông tin cụ thể, hãy liên hệ tương ứng với thắc mắc của bạn:")
                for contact in related_contacts:
                    print(f"- {contact}: {hangmuc_to_contact[contact]}")

            if debug:
                print("\nThông tin các đoạn context:")
                for doc in docs:
                    print(f"Context từ file: {doc.metadata['filename']}")
                    print(f"Điều: {doc.metadata['sourcename']}")
                    print(f"Score: {doc.metadata['score']:.2f}")
                    print(f"Nội dung: {doc.page_content}\n")
            return answer  # Trả về để xử lý tiếp nếu cần

    # Nếu không tìm được câu trả lời nào phù hợp
    if debug:
        print("RAG không đủ thông tin. Đang tìm trên internet...")
    try:
        search_tool = DuckDuckGoSearchRun() # Tra ve 1 cau hoi, neu muon tra ve nhieu cau tra loi, phai dung DuckDuckGoSearchResults
        tools = [
            Tool(
                name="Web Search",
                func=search_tool.run,
                description="Search the internet when no answer can be found in documents"
            )
        ]
        agent = initialize_agent(tools, conversation.llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose = False)
        modified_question = f"{question}"
        web_result = agent.run(modified_question)
        print(f"Câu trả lời từ web: {web_result} \n Câu trả lời này không được đảm bảo chính xác. Bạn nên kiểm tra lại thông tin ở nguồn chính thức.")

        # Cập nhật vào history thông qua conversation
        conversation.memory.chat_memory.add_user_message(HumanMessage(content=question))
        conversation.memory.chat_memory.add_ai_message(AIMessage(content=web_result))
    except Exception as e:
        web_result = "Tôi xin lỗi, tôi không thể tìm thấy thông tin trên internet."

    return web_result


In [31]:
# Bao nhiêu điểm là đạt học phần trong hệ thống tín chỉ? # truong hop reformulation query giup tra loi cau hoi
question = "Tôi là sinh viên năm hai, đang bị cảnh báo học tập mức 2 và có 26 tín chỉ nợ đọng, tôi sẽ bị áp dụng những hình thức nào theo quy chế?"
question = "Giải thích vì sao chương trình tích hợp cử nhân – kỹ sư lại có khối lượng 180 tín chỉ và thời gian thiết kế 5,5 năm?"
question = "Nếu em đang học năm cuối mà lỡ bị cảnh cáo lần hai do quên đăng ký học kỳ hè, liệu có khả năng bị đuổi học không? Quy chế có linh động gì không?"
question = "Đại học Kinh tế Quốc dân có bao nhiêu ngành học ?"
question = "Giả sử em bị điểm liệt đồ án tốt nghiệp nhưng thầy hướng dẫn lại đánh giá cao, không biết có cách nào cứu vãn điểm đó không hay là mặc định bị F luôn?"
question = "Nếu em có bài báo khoa học mà không liên quan chặt tới luận văn, liệu có được tính điểm cộng khi bảo vệ không? Hay bắt buộc phải đúng chủ đề?"
question = """Trong trường hợp tụi em đăng ký lớp mà chưa đủ 20 người, thì khả năng cao là lớp đó bị hủy đúng không? Hay vẫn có ngoại lệ nếu là lớp ngành "hiếm"?"""
question = """Em có một học phần đã học và đạt điểm cao, nhưng sau này chương trình có cập nhật lại danh mục môn học bắt buộc, khiến học phần đó không còn nằm trong danh sách quy định nữa. Vậy học phần đó có bị loại khỏi CPA không? Có được xem là học phần tự chọn tích lũy không? Có trường hợp nào bắt buộc phải học lại không?"""
question = """Nếu em từng bị buộc thôi học do học lực kém, sau đó thi lại và trúng tuyển ngành khác trong cùng trường, thì các học phần đã từng học có được công nhận lại không? Nếu có thì có bị giới hạn số tín chỉ hay thời gian sử dụng không? Em có thể dùng lại chứng chỉ Giáo dục quốc phòng cũ không?"""
question = "Em từng chuyển từ hệ chính quy sang vừa học vừa làm do điều kiện cá nhân, hiện đã học xong gần hết chương trình nhưng nhận ra một số học phần bị thay đổi mã số và nội dung so với thời điểm em học hệ chính quy, vậy các học phần đã tích lũy đó có được bảo lưu đầy đủ và tính vào điều kiện tốt nghiệp không?"
question = """Kính gửi thầy, cô.
Em là Trần Tùng Lâm 20210507. Đầu kì này em có đăng kí học phần project III, và được đăng kí học phần IT3940. Đến hôm qua em mới biết là bản thân đăng kí nhầm, ngành Kỹ Thuật Máy Tính yêu cầu project3 là IT3943.
Em xin hỏi có được quy đổi tương đương không ạ. Nếu không được thì liệu 2025.1 hoặc kì 2024.3 em có thể đăng kí được học phần IT3943 không ạ. Em hiện đang hoang mang vì nếu không quy đổi được và 2025.1 cũng như 2024.3 không mở đăng kí thì em k đăng kí được Đồ Án Tốt Nghiệp ạ.
Em mong được thầy cô tư vấn ạ.
Sinh viên
Trần Tùng Lâm IT2-K66"""
#question = """"Em tốt nghiệp với CPA là 3.6 và đã học cải thiện điểm 7 tín, học lại 5 tín do bị nợ môn. Chương trình đào tạo của em cần hoàn thành 138 tín chỉ. Vậy em được tốt nghiệp bằng loại gì ?""""


summary_memory = ConversationSummaryMemory(llm=llm, return_messages=True)
conversation = ConversationChain(llm=llm, memory=summary_memory, verbose = True) # neu muon xem cac history thi de verbose = True

#ask_question_with_variants(question, custom_bm25_retriever, llm, template, all_chunks)

# Danh sách các câu hỏi cố định
questions = [
    "Đại học kinh tế quốc dân có bao nhiêu ngành học ?"

]

# Vòng lặp xử lý từng câu hỏi
for idx, question in enumerate(questions, 1):
    print(f"\n================ Câu hỏi {idx} ================\n")
    answer = ask_question_with_variants(
        question=question,
        retriever=custom_bm25_retriever,
        conversation=conversation,
        template=template,
        all_chunks=all_chunks,
    )
    print(f"Trợ lý: {answer}")
    if answer == "Tôi xin lỗi, tôi không thể tìm thấy thông tin trên internet.":
        related_contacts = get_best_related_contact(question, model, hangmuc_embeddings,5)
        print("Nếu bạn cần thông tin cụ thể, hãy liên hệ tương ứng với thắc mắc của bạn:")
        for contact in related_contacts:
            print(f"- {contact}: {hangmuc_to_contact[contact]}")





================ Câu hỏi 1 ================

Các câu hỏi đã tạo:
1. Đại học kinh tế quốc dân có bao nhiêu ngành học ?
2. Đại học kinh tế quốc dân đào tạo bao nhiêu ngành học?
3. Số lượng ngành học tại Đại học kinh tế quốc dân là bao nhiêu?

Đang tìm kiếm câu trả lời từ RAG...


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
[SystemMessage(content='', additional_kwargs={}, response_metadata={})]
Human: <|im_start|>system
Bạn là một chatbot trả lời về quy định của Đại học Bách khoa Hà Nội. Chỉ trả lời dựa trên thông tin đã cung cấp và nêu rõ dựa trên thông tin gì để trả lời. Nếu không chắc chắn hoặc không tìm thấy, hãy trả lời 'Tôi xin lỗi, tôi không có đủ thông tin để trả lời câu hỏi này.'
Đoạn context này thuộc điều

In [29]:
print("--------- Lịch sử hội thoại ---------")
for msg in conversation.memory.chat_memory.messages:
    prefix = "Người dùng:" if msg.type == "human" else "Trợ lý:"
    print(f"{prefix} {msg.content}")
print("-------------------------------------\n")


--------- Lịch sử hội thoại ---------
Người dùng: <|im_start|>system
Bạn là một chatbot trả lời về quy định của Đại học Bách khoa Hà Nội. Chỉ trả lời dựa trên thông tin đã cung cấp và nêu rõ dựa trên thông tin gì để trả lời. Nếu không chắc chắn hoặc không tìm thấy, hãy trả lời 'Tôi xin lỗi, tôi không có đủ thông tin để trả lời câu hỏi này.'
Đoạn context này thuộc điều 33:Điều 33 . Điều kiện tốt nghiệp thạc sĩ và xếp hạng tốt nghiệp 1 . Học viên có đủ các điều kiện sau đây thì được xét công nhận tốt nghiệp : a ) Luận văn đạt yêu cầu . b ) Hoàn thành việc nộp quyển luận văn đã được chỉnh sửa theo kết luận của Hội đồng đánh giá luận văn . Công khai toàn văn quyển luận văn trên website theo quy định trong thời gian ít nhất 30 ngày , trừ một số đề tài thuộc các lĩnh vực cần bảo mật thực hiện theo quy định của Nhà nước . c ) Đã nộp quyển luận văn tốt nghiệp cho thư viện theo quy cách của bản luận văn do ĐHBK Hà Nội quy định . d ) Không bị truy cứu trách nhiệm hình sự và không trong thời gian